In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0008}
band = [6]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.66, 1.66, 1.85)]
  wgt = [(0.5, 0.55, 0.85)]
o co2 band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.000127                    1 -0.000255
0.000750        2 -0.000060                    2 -0.000318
0.001052        3  0.000107                    3 -0.000049
0.001476        4  0.000330                    4  0.000106
0.002070        5  0.000617                    5  0.000296
0.002904        6  0.000982                    6  0.000525
0.004074        7  0.001439                    7  0.000831
0.005714        8  0.002006                    8  0.001239
0.008015        9  0.002704                    9  0.001739
0.011243       10  0.003556                   10  0.002339
0.015771       11  0.004589                   11  0.003057
0.022122       12  0.005835                   12  0.003917
0.031031       13  0.007322                   13  0.004940
0.043528       14  0.008864                   14  0.006000
0.061057       15  0.010207                   15  0.006940
0.085645       16  0.011482                   16  0.007847
0.120136       17  0.012863                   17  0.008822
0.168516       18  0.014358                   18  0.009849
0.236378       19  0.015977                   19  0.010919
0.331549       20  0.017719                   20  0.012039
0.465100       21  0.019384                   21  0.013218
0.652400       22  0.017752                   22  0.012445
0.915100       23  0.012618                   23  0.009098
1.283650       24  0.008020                   24  0.005924
1.800600       25  0.004850                   25  0.003657
2.525700       26  0.002723                   26  0.002095
3.542800       27  0.001343                   27  0.001045
4.969550       28  0.000569                   28  0.000433
6.970850       29  0.000192                   29  0.000113
9.778100       30 -0.000027                   30 -0.000089
13.715850      31 -0.000184                   31 -0.000241
19.239350      32 -0.000294                   32 -0.000352
26.987250      33 -0.000336                   33 -0.000404
37.855300      34 -0.000326                   34 -0.000408
53.100050      35 -0.000314                   35 -0.000428
73.887500      36 -0.000313                   36 -0.000448
97.662500      37 -0.000328                   37 -0.000453
121.437500     38 -0.000370                   38 -0.000466
145.212500     39 -0.000426                   39 -0.000481
168.987500     40 -0.000476                   40 -0.000493
192.762500     41 -0.000522                   41 -0.000506
216.537500     42 -0.000564                   42 -0.000531
240.312500     43 -0.000602                   43 -0.000559
264.087500     44 -0.000638                   44 -0.000584
287.862500     45 -0.000665                   45 -0.000599
311.637500     46 -0.000631                   46 -0.000558
335.412500     47 -0.000486                   47 -0.000408
359.187500     48 -0.000237                   48 -0.000170
382.962500     49  0.000098                   49  0.000143
406.737500     50  0.000544                   50  0.000550
430.512500     51  0.001129                   51  0.001066
454.287500     52  0.001884                   52  0.001727
478.062500     53  0.002838                   53  0.002572
501.837500     54  0.004018                   54  0.003629
525.612500     55  0.005438                   55  0.004849
549.387500     56  0.007095                   56  0.006303
573.162500     57  0.008972                   57  0.007992
596.937500     58  0.011029                   58  0.009894
620.712500     59  0.013210                   59  0.011970
644.487500     60  0.015452                   60  0.014097
668.262500     61  0.017692                   61  0.016213
692.037500     62  0.019874                   62  0.018325
715.812500     63  0.021956                   63  0.020387
739.587500     64  0.023908                   64  0.022366
763.362500     65  0.025719                   65  0.024238
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -32.681579  0.000000e+00 -32.681579 -32.697067   
0.000624    2     -32.681579  6.993917e-08 -32.681579 -32.697067   
0.000876    3     -32.681579  1.008145e-07 -32.681579 -32.697067   
0.001229    4     -32.681579  1.528364e-07 -32.681579 -32.697067   
0.001723    5     -32.681579  2.405276e-07 -32.681579 -32.697067   
0.002417    6     -32.681579  3.879834e-07 -32.681579 -32.697067   
0.003391    7     -32.681579  6.354104e-07 -32.681579 -32.697067   
0.004757    8     -32.681579  1.049622e-06 -32.681578 -32.697067   
0.006672    9     -32.681580  1.741293e-06 -32.681578 -32.697068   
0.009359    10    -32.681580  2.893805e-06 -32.681577 -32.697068   
0.013128    11    -32.681580  4.810104e-06 -32.681575 -32.697068   
0.018415    12    -32.681581  7.989727e-06 -32.681573 -32.697069   
0.025830    13    -32.681581  1.325452e-05 -32.681568 -32.697069   
0.036232    14    -32.681580  2.194772e-05 -32.681558 -32.697069   
0.050823    15    -32.681579  3.599443e-05 -32.681543 -32.697069   
0.071291    16    -32.681576  5.790029e-05 -32.681518 -32.697068   
0.100000    17    -32.681571  9.149205e-05 -32.681479 -32.697066   
0.140271    18    -32.681561  1.428605e-04 -32.681418 -32.697062   
0.196760    19    -32.681543  2.211852e-04 -32.681322 -32.697053   
0.275997    20    -32.681512  3.402308e-04 -32.681172 -32.697037   
0.387100    21    -32.681459  5.203309e-04 -32.680939 -32.697009   
0.543100    22    -32.681370  7.893663e-04 -32.680581 -32.696963   
0.761700    23    -32.681255  1.134134e-03 -32.680121 -32.696905   
1.068500    24    -32.681155  1.493156e-03 -32.679662 -32.696866   
1.498800    25    -32.681092  1.838724e-03 -32.679254 -32.696861   
2.102400    26    -32.681074  2.166988e-03 -32.678907 -32.696899   
2.949000    27    -32.681109  2.475542e-03 -32.678634 -32.696988   
4.136600    28    -32.681209  2.764331e-03 -32.678445 -32.697138   
5.802500    29    -32.681383  3.050393e-03 -32.678332 -32.697359   
8.139200    30    -32.681647  3.368108e-03 -32.678279 -32.697673   
11.417000   31    -32.682036  3.746086e-03 -32.678290 -32.698112   
16.014700   32    -32.682598  4.207930e-03 -32.678390 -32.698723   
22.464000   33    -32.683406  4.792068e-03 -32.678614 -32.699576   
31.510500   34    -32.684591  5.616636e-03 -32.678974 -32.700787   
44.200100   35    -32.686374  6.909069e-03 -32.679465 -32.702540   
62.000000   36    -32.689080  8.952795e-03 -32.680127 -32.705261   
85.775000   37    -32.692993  1.198527e-02 -32.681008 -32.709321   
109.550000  38    -32.697178  1.524748e-02 -32.681931 -32.713661   
133.325000  39    -32.701564  1.859188e-02 -32.682973 -32.718134   
157.100000  40    -32.706122  2.194937e-02 -32.684172 -32.722617   
180.875000  41    -32.710833  2.531913e-02 -32.685514 -32.727111   
204.650000  42    -32.715689  2.870465e-02 -32.686985 -32.731643   
228.425000  43    -32.720675  3.210263e-02 -32.688572 -32.736323   
252.200000  44    -32.725781  3.551356e-02 -32.690268 -32.741182   
275.975000  45    -32.731007  3.894208e-02 -32.692065 -32.746188   
299.750000  46    -32.736367  4.242958e-02 -32.693938 -32.751331   
323.525000  47    -32.741972  4.625618e-02 -32.695716 -32.756704   
347.300000  48    -32.747961  5.087713e-02 -32.697084 -32.762345   
371.075000  49    -32.754349  5.659723e-02 -32.697752 -32.768307   
394.850000  50    -32.761191  6.371424e-02 -32.697477 -32.774678   
418.625000  51    -32.768536  7.259272e-02 -32.695943 -32.781463   
442.400000  52    -32.776391  8.362825e-02 -32.692763 -32.788607   
466.175000  53    -32.784748  9.729145e-02 -32.687457 -32.796152   
489.950000  54    -32.793575  1.141120e-01 -32.679463 -32.804102   
513.725000  55    -32.802805  1.346584e-01 -32.668146 -32.812416   
537.500000  56    -32.812335  1.595026e-01 -32.652832 -32.820885   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')